In [1]:
import sys
sys.executable

'/mnt/c/users/nawfal/documents/apps/collaborative project/cctv_motion_detection/motion_venv/bin/python'

In [2]:
import cv2
import os
import time
import numpy as np
import csv

In [3]:
# cap = cv2.VideoCapture("http://192.168.50.234:5000/video")
cap = cv2.VideoCapture("http://192.168.100.197:5000/video")


In [ ]:
# Initialize the camera

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

# Capture frames from the camera
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    # Display the frame
    cv2.namedWindow("Camera Capture", cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow("Camera Capture", 640, 480)
    cv2.imshow("Camera Capture", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Import Mediapipe

In [5]:
import cv2
import os
import time
import csv
import warnings
import pickle

import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


# Z Coordinate detection
Zoom in & Zoom Out while move forward and backwards

In [ ]:
lndmrkX = None
lndmrkY = None
lndmrkZ = None

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic :

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Get specific landmarks
        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]

        show_landmark_list = landmark_pb2.NormalizedLandmarkList()
        show_landmark_list.landmark.extend([nose])

        # Draw landmarks
        for landmark in show_landmark_list.landmark:
            x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
            lndmrkX = landmark.x
            lndmrkY = landmark.y
            lndmrkZ = landmark.z

        cv2.circle(image, (x, y), 5, (255, 0, 0), -1)

        cv2.putText(image,
            f"Nose: (\nz: {lndmrkZ:.2f})",
            (x + 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,               # font scale
            (0, 255, 0),       # color
            2,                 # thickness
            cv2.LINE_AA)
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

move forward goes till minus

# Face Direction Detection

In [ ]:
def check_direction(landmark):
    direction = "Right"
    smallest_value = min(landmark, key=landmark.get)
    highest_value = max(landmark, key=landmark.get)
    
    if smallest_value == "nose":
        direction = "Right"
    elif highest_value == "nose":
        direction = "Left"
    else:
        direction = "Center"
    
    return direction

    

In [33]:
camera = cv2.VideoCapture("http://192.168.100.197:5000/video")

# Check if the camera opened successfully
if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.1, min_tracking_confidence=0.1) as holistic :
    
    while camera.isOpened():  
        ret, frame = camera.read()

        if not ret:
            print("❌ Failed to grab frame")
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # =============== THE LANDMARKS ============== #
        
        nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
        ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
        ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
        wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

        # Array Each Landmark
        face_direction = check_direction({"nose": nose.x, "ear_r": ear_r.x, "ear_l": ear_l.x}) 

        # For Draw
        lm = landmark_pb2.NormalizedLandmarkList()
        lm.landmark.extend([nose, ear_r, ear_l, wrist_r])

        # ============================================ #

        for landmark in lm.landmark:
            x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
            cv2.circle(image, (x, y), 5, (255, 0, 0), -1)
        
        cv2.putText(image,
            # f"x: ({landmark.x:.2f})",
            f"x: ({face_direction})",
            (x + 10, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,               # font scale
            (0, 255, 0),       # color
            2,                 # thickness
            cv2.LINE_AA)

        # =============== DETECTION ================== #
        # ============================================ #

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

camera.release()
cv2.destroyAllWindows()

Camera is ready. Press 'q' to quit.


I0000 00:00:1755499288.940323  160903 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1755499288.970242  174902 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
W0000 00:00:1755499289.563085  174891 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755499289.642083  174893 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755499289.654893  174891 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755499289.660571  174898 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00

Left & Right Eye Can Be Used For the Left and Right